# Multi-Class Classification

This lesson picks up where <a href="https://github.com/4igeek/TensorFlow/blob/main/Classification/NonLinearBinaryClassification.ipynb">Non-Linear Binary Classification</a> takes off. In this notebook we're going to look at solving another classification problem only this time we're going to have more than two labels (classes). The steps for solving a binary class problem are the same as the steps for solving a multi-class problem.

In this example we are going to classify images of clothing i.e. t-shirts, shoes etc.

## Recap of the steps for building a model
1) Prepare the data
2) Build a model
3) Fit the model to the data (and make predictions)
4) Evaluate the model
5) Improve through experimentation
6) Save the model

## 1) Prepare the data
